In [7]:
import sys
sys.path.append('/Users/yuhu/Desktop/p4-Speech/Speech-Speaker-Recognition/lab1')
sys.path.append('/Users/yuhu/Desktop/p4-Speech/Speech-Speaker-Recognition/lab2')
from lab1_proto import *
from lab2_proto import *
from tqdm import trange
import os
import numpy as np 
import re
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import PReLU
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import Sequence
from tensorflow.keras.backend import one_hot
from sklearn.preprocessing import StandardScaler

In [8]:
a = np.arange(9).reshape((3,3))
scaler = StandardScaler()
scaler.fit(a)
print(a)
print(scaler.transform(a))

[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[-1.22474487 -1.22474487 -1.22474487]
 [ 0.          0.          0.        ]
 [ 1.22474487  1.22474487  1.22474487]]


In [9]:
phoneHMMs_all = np.load('/Users/yuhu/Desktop/p4-Speech/Speech-Speaker-Recognition/lab2/lab2_models_all.npz',allow_pickle=True)['phoneHMMs'].item()
phones = sorted(phoneHMMs_all.keys())
nstates = {phone: phoneHMMs_all[phone]['means'].shape[0] for phone in phones}
stateList = [ph + '_' + str(i) for ph in phones for i in range(nstates[ph])]

In [10]:
traindata=np.load('train_nondynamic.npz',allow_pickle=1)
X_train_lmfcc = traindata['lmfcc']
X_train_mspec = traindata['mspec']
Y_train = traindata['Y']
Y_train = np.squeeze(one_hot(Y_train,len(stateList)))

valdata=np.load('val_nondynamic.npz',allow_pickle=1)
X_val_lmfcc = valdata['lmfcc']
X_val_mspec = valdata['mspec']
Y_val = valdata['Y']
Y_val = np.squeeze(one_hot(Y_val,len(stateList)))

testdata=np.load('test_nondynamic.npz',allow_pickle=1)
X_test_lmfcc = testdata['lmfcc']
X_test_mspec = testdata['mspec']
Y_test = testdata['Y']
Y_test = np.squeeze(one_hot(Y_test,len(stateList)))

traindata_d=np.load('train_dynamic.npz',allow_pickle=1)
X_train_lmfcc_d = traindata_d['lmfcc']
X_train_mspec_d = traindata_d['mspec']
Y_train_d = traindata_d['Y']
Y_train_d = np.squeeze(one_hot(Y_train_d,len(stateList)))

valdata_d=np.load('val_dynamic.npz',allow_pickle=1)
X_val_lmfcc_d = valdata_d['lmfcc']
X_val_mspec_d = valdata_d['mspec']
Y_val_d = valdata_d['Y']
Y_val_d = np.squeeze(one_hot(Y_val_d,len(stateList)))

testdata_d=np.load('test_dynamic.npz',allow_pickle=1)
X_test_lmfcc_d = testdata_d['lmfcc']
X_test_mspec_d = testdata_d['mspec']
Y_test_d = testdata_d['Y']
Y_test_d = np.squeeze(one_hot(Y_test_d,len(stateList)))

In [11]:
def normalize(X_train,X_val,X_test): 
  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train=scaler.transform(X_train)
  X_val=scaler.transform(X_val)
  X_test=scaler.transform(X_test)

  return X_train,X_val,X_test

In [12]:
X_train_lmfcc,X_val_lmfcc,X_test_lmfcc=normalize(X_train_lmfcc,X_val_lmfcc,X_test_lmfcc)
X_train_mspec_d,X_val_mspec_d,X_test_mspec_d=normalize(X_train_mspec_d,X_val_mspec_d,X_test_mspec_d)
X_train_mspec,X_val_mspec,X_test_mspec=normalize(X_train_mspec,X_val_mspec,X_test_mspec)
X_train_lmfcc_d,X_val_lmfcc_d,X_test_lmfcc_d=normalize(X_train_lmfcc_d,X_val_lmfcc_d,X_test_lmfcc_d)

In [13]:
def get_model(input_shape,n_states):
  Inp = Input(input_shape)
  x = Dense(256,activation='relu')(Inp)
  x = Dense(256,activation='relu')(x)
  x = Dense(256,activation='relu')(x)
  y = Dense(n_states,activation='softmax')(x)
  model = Model(inputs=Inp,outputs=y)
  print(model.summary())
  return model

In [14]:
model_lmfcc = get_model(X_train_lmfcc[-1].shape,len(stateList))
model_lmfcc.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 13)]              0         
_________________________________________________________________
dense (Dense)                (None, 256)               3584      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 61)                15677     
Total params: 150,845
Trainable params: 150,845
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
his_lmffc = model_lmfcc.fit(X_train_lmfcc,Y_train,batch_size=256,epochs=20)


Train on 1487243 samples
Epoch 1/20
1487243/1487243 [==============================] - 16s 10us/sample - loss: 1.7210 - accuracy: 0.4904
Epoch 2/20
1487243/1487243 [==============================] - 15s 10us/sample - loss: 1.5949 - accuracy: 0.5167
Epoch 3/20
1487243/1487243 [==============================] - 30s 20us/sample - loss: 1.5570 - accuracy: 0.5259
Epoch 4/20
1487243/1487243 [==============================] - 26s 17us/sample - loss: 1.5324 - accuracy: 0.5319
Epoch 5/20
1487243/1487243 [==============================] - 17s 11us/sample - loss: 1.5142 - accuracy: 0.5362
Epoch 6/20
1487243/1487243 [==============================] - 18s 12us/sample - loss: 1.5008 - accuracy: 0.5396
Epoch 7/20
1487243/1487243 [==============================] - 22s 15us/sample - loss: 1.4891 - accuracy: 0.5430
Epoch 8/20
1487243/1487243 [==============================] - 21s 14us/sample - loss: 1.4797 - accuracy: 0.5456
Epoch 9/20
1487243/1487243 [==============================] - 23s 15us/sample -

In [17]:
model_lmfcc_d = get_model(X_train_lmfcc_d[-1].shape,len(stateList))
model_lmfcc_d.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 91)]              0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               23552     
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_6 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_7 (Dense)              (None, 61)                15677     
Total params: 170,813
Trainable params: 170,813
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
his_lmffc_d = model_lmfcc_d.fit(X_train_lmfcc_d,Y_train_d,batch_size=256,epochs=20)

Train on 1487243 samples
Epoch 1/20
1487243/1487243 [==============================] - 17s 12us/sample - loss: 0.7769 - accuracy: 0.7441
Epoch 2/20
1487243/1487243 [==============================] - 16s 11us/sample - loss: 0.6086 - accuracy: 0.7915
Epoch 3/20
1487243/1487243 [==============================] - 15s 10us/sample - loss: 0.5628 - accuracy: 0.8061
Epoch 4/20
1487243/1487243 [==============================] - 14s 10us/sample - loss: 0.5363 - accuracy: 0.8146
Epoch 5/20
1487243/1487243 [==============================] - 15s 10us/sample - loss: 0.5181 - accuracy: 0.8204
Epoch 6/20
1487243/1487243 [==============================] - 17s 11us/sample - loss: 0.5044 - accuracy: 0.8245
Epoch 7/20
1487243/1487243 [==============================] - 21s 14us/sample - loss: 0.4939 - accuracy: 0.8279
Epoch 8/20
1487243/1487243 [==============================] - 19s 13us/sample - loss: 0.4846 - accuracy: 0.8307
Epoch 9/20
1487243/1487243 [==============================] - 17s 11us/sample -

In [25]:
model_mspec = get_model(X_train_mspec[-1].shape,len(stateList))
model_mspec.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 40)]              0         
_________________________________________________________________
dense_20 (Dense)             (None, 256)               10496     
_________________________________________________________________
dense_21 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_22 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_23 (Dense)             (None, 61)                15677     
Total params: 157,757
Trainable params: 157,757
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
his_mspec = model_mspec.fit(X_train_mspec,Y_train,batch_size=256,epochs=20)


Train on 1487243 samples
Epoch 1/20
1487243/1487243 [==============================] - 16s 10us/sample - loss: 1.7238 - accuracy: 0.4901
Epoch 2/20
1487243/1487243 [==============================] - 14s 9us/sample - loss: 1.5519 - accuracy: 0.5279
Epoch 3/20
1487243/1487243 [==============================] - 13s 9us/sample - loss: 1.4954 - accuracy: 0.5418
Epoch 4/20
1487243/1487243 [==============================] - 14s 9us/sample - loss: 1.4613 - accuracy: 0.5506
Epoch 5/20
1487243/1487243 [==============================] - 14s 9us/sample - loss: 1.4376 - accuracy: 0.5563
Epoch 6/20
1487243/1487243 [==============================] - 14s 10us/sample - loss: 1.4196 - accuracy: 0.5610
Epoch 7/20
1487243/1487243 [==============================] - 14s 9us/sample - loss: 1.4049 - accuracy: 0.5649
Epoch 8/20
1487243/1487243 [==============================] - 14s 9us/sample - loss: 1.3926 - accuracy: 0.5679
Epoch 9/20
1487243/1487243 [==============================] - 16s 11us/sample - loss:

In [28]:
model_mspec_d = get_model(X_train_mspec_d[-1].shape,len(stateList))
model_mspec_d.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 280)]             0         
_________________________________________________________________
dense_28 (Dense)             (None, 256)               71936     
_________________________________________________________________
dense_29 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_30 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_31 (Dense)             (None, 61)                15677     
Total params: 219,197
Trainable params: 219,197
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
his_mspec_d = model_mspec.fit(X_train_mspec_d,Y_train,batch_size=256,epochs=20)
